# 第7章: 単語ベクトル
単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．
60. 単語ベクトルの読み込みと表示
61. 単語の類似度
62. 類似度の高い単語10件
63. 加法構成性によるアナロジー
64. アナロジーデータでの実験
65. アナロジータスクでの正解率
66. WordSimilarity-353での評価
67. k-meansクラスタリング
68. Ward法によるクラスタリング
69. t-SNEによる可視化


60. 単語ベクトルの読み込みと表示<br>
Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル（300万単語・フレーズ，300次元）](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import KeyedVectors

# 1. Google Newsで学習済みWord2Vecモデルをダウンロードし保存
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM -O /content/drive/MyDrive/Colab Notebooks/data/GoogleNews-vectors-negative300.bin.gz

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=862249d8-872b-4f97-99db-4973ce129bfc
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:19<00:00, 86.0MB/s]


In [4]:
# 2. モデルのロード（バイナリ形式で読み込み）
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/data/GoogleNews-vectors-negative300.bin.gz', binary=True)

# 3. "United States"の単語ベクトルを取得
vector = model['United_States']

# 4. 結果表示
print("United States の単語ベクトル（最初の10次元）:")
print(vector[:10])

NameError: name 'KeyedVectors' is not defined

61. 単語の類似度<br>
“United States”と”U.S.”のコサイン類似度を計算せよ．

In [ ]:
# Google Newsで学習済みのWord2Vecモデルを使って類似度計算
similarity = model.similarity('United_States', 'U.S.')
print(similarity)

0.73107743


62. 類似度の高い単語10件<br>
“United States”とコサイン類似度が高い10語と，その類似度を出力せよ

In [ ]:
# Google Newsで学習済みのWord2Vecモデルを使って類似度計算
similar_words = model.most_similar('United_States', topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity: .4f}")

Unites_States:  0.7877
Untied_States:  0.7541
United_Sates:  0.7401
U.S.:  0.7311
theUnited_States:  0.6404
America:  0.6178
UnitedStates:  0.6167
Europe:  0.6133
countries:  0.6045
Canada:  0.6019


63. 加法構成性によるアナロジー<br>
“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [ ]:
Spain = model['Spain']
Madrid = model['Madrid']
Athens = model['Athens']

# Madrid - Spain + Athens
result = Madrid - Spain + Athens

# Google Newsで学習済みのWord2Vecモデルを使ってresultベクトルから類似度計算
similar_words = model.similar_by_vector(result, topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity: .4f}")

Athens:  0.8219
Madrid:  0.5877
Rome:  0.5468
Athens_Greece:  0.5300
Peania:  0.4843
Athen:  0.4821
Mykonos_Island:  0.4807
Cairo:  0.4799
Organizing_Committee_ATHOC:  0.4790
Thessaloniki:  0.4779


64. アナロジーデータでの実験<br>
単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

In [ ]:
!wget -P /content/drive/MyDrive/Colab\ Notebooks/data http://download.tensorflow.org/data/questions-words.txt

--2024-12-29 05:27:44--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.11.207, 108.177.12.207, 74.125.26.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.11.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘/content/drive/MyDrive/Colab Notebooks/data/questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.01s   

2024-12-29 05:27:44 (41.6 MB/s) - ‘/content/drive/MyDrive/Colab Notebooks/data/questions-words.txt’ saved [603955/603955]



In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/Colab Notebooks/data")
# データを格納するリスト
results = []

# ファイルを読み込んで処理
with open(data_dir / 'questions-words.txt', 'r') as f:
    for line in f:
        if line.startswith(':'):
            current_category = line.strip().replace(':', '')
        else:
            word1, word2, word3, word4 = line.strip().split()
            # Google Newsで学習済みのWord2Vecモデルを使って類似度計算
            try:
                result = model.most_similar(positive=[word2, word3], negative=[word1], topn=1)
                predicted_word, similarity = result[0]
            except KeyError:
                predicted_word, similarity = 'NONE', 0.

            # 結果をリストに追加
            results.append({
                'category': current_category,
                'word1': word1,
                'word2': word2,
                'word3': word3,
                'word4': word4,
                'predicted': predicted_word,
                'similarity': similarity
            })

# DataFrameに変換
df = pd.DataFrame(results)
# CSVに保存
csv_path = data_dir / 'questions-words-with-results.csv'
df.to_csv(csv_path, index=False)
print(f"CSVに保存しました: {csv_path}")

CSVに保存しました: /content/drive/MyDrive/Colab Notebooks/data/questions-words-with-results.csv


65. アナロジータスクでの正解率<br>
64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

In [ ]:
df = pd.read_csv(data_dir / 'questions-words-with-results.csv')
print(df.head())

def categorize(row):
    if 'gram' in row['category'].lower():
        return 'syntactic'
    else:
        return 'semantic'

df['type'] = df.apply(categorize, axis=1)

type_accuracy = df.groupby('type').apply(lambda x: (x['word4'] == x['predicted']).sum() / len(x), include_groups=False)

# 結果を表示
print("\n意味的アナロジーと文法的アナロジーの正解率:")
print(type_accuracy)

                    category   word1   word2    word3        word4  \
0   capital-common-countries  Athens  Greece  Baghdad         Iraq   
1   capital-common-countries  Athens  Greece  Bangkok     Thailand   
2   capital-common-countries  Athens  Greece  Beijing        China   
3   capital-common-countries  Athens  Greece   Berlin      Germany   
4   capital-common-countries  Athens  Greece     Bern  Switzerland   

     predicted  similarity  
0        Iraqi    0.635187  
1     Thailand    0.713767  
2        China    0.723578  
3      Germany    0.673462  
4  Switzerland    0.491975  

意味的アナロジーと文法的アナロジーの正解率:
type
semantic     0.730860
syntactic    0.740047
dtype: float64


66. WordSimilarity-353での評価<br>
The WordSimilarity-353 Test Collectionの評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ

In [ ]:
!wget -P data_dir https://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip

--2024-12-29 08:48:16--  https://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving www.gabrilovich.com (www.gabrilovich.com)... 173.236.137.139
Connecting to www.gabrilovich.com (www.gabrilovich.com)|173.236.137.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257 (23K) [application/zip]
Saving to: ‘data_dir/wordsim353.zip’

wordsim353.zip      100%[===================>]  22.71K  --.-KB/s    in 0.02s   

2024-12-29 08:48:16 (1.23 MB/s) - ‘data_dir/wordsim353.zip’ saved [23257/23257]



In [ ]:
import zipfile

data_path = data_dir / "wordsim353.zip"
extract_path = data_dir / "wordsim353"

# ZIPファイルの解凍
with zipfile.ZipFile(data_path, 'r') as zip_red:
    zip_red.extractall(extract_path)

print("解凍完了")
!ls data_dir / "wordsim353"

解凍完了
ls: cannot access 'wordsim353': No such file or directory
/:
bin			    dev    lib64		     opt		run   tools
boot			    etc    libx32		     proc		sbin  usr
content			    home   media		     python-apt		srv   var
cuda-keyring_1.0-1_all.deb  lib    mnt			     python-apt.tar.xz	sys
datalab			    lib32  NGC-DL-CONTAINER-LICENSE  root		tmp

data_dir:
wordsim353.zip


In [ ]:
from scipy.stats import spearmanr

# CSVの読み込み
df = pd.read_csv(data_dir / 'wordsim353/combined.csv')

results = []

for _, row in df.iterrows():
    word1 = row['Word 1']
    word2 = row['Word 2']
    human_similarity = row['Human (mean)']

    try:
        # Google Newsで学習済みのWord2Vecモデルを使って類似度計算
        model_similarity = model.similarity(word1, word2)
    except KeyError:
        model_similarity = 0.

    results.append({
        'word1': word1,
        'word2': word2,
        'human_similarity': human_similarity,
        'model_similarity': model_similarity
    })

result_df = pd.DataFrame(results)

print(result_df.head())

spearman_corr, _ = spearmanr(result_df['human_similarity'], result_df['model_similarity'])

print(f"スピアマン相関係数: {spearman_corr:.4f}")

      word1     word2  human_similarity  model_similarity
0      love       sex              6.77          0.263938
1     tiger       cat              7.35          0.517296
2     tiger     tiger             10.00          1.000000
3      book     paper              7.46          0.363463
4  computer  keyboard              7.62          0.396392
スピアマン相関係数: 0.7000


67. k-meansクラスタリング<br>
国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．

In [ ]:
from sklearn.cluster import KMeans

# modelはロード済のGoogle Newsで学習済みのWord2Vecモデル

# 1. 国名リストの準備
countries = [
    'Japan', 'Germany', 'France', 'Italy', 'Brazil', 'Canada', 'India', 'China',
    'Russia', 'Spain', 'Mexico', 'Netherlands', 'Turkey', 'Australia', 'Sweden',
    'Norway', 'Argentina', 'South_Korea', 'United_States', 'Egypt', 'Nigeria',
    'South_Africa', 'Pakistan', 'Iran', 'Iraq', 'Israel', 'Vietnam', 'Thailand',
    'Poland', 'Greece', 'Portugal', 'Malaysia', 'Singapore', 'Denmark', 'Finland'
]

# 2. 国名の単語ベクトルを取得
vectors = []
valid_countries = []

for country in countries:
    try:
        vectors.append(model[country])
        valid_countries.append(country)  # モデルに存在する国だけリストに追加
    except KeyError:
        pass  # モデルに存在しない国はスキップ

print(f"モデルに存在する国数: {len(valid_countries)}")

# 3. k-meansクラスタリング (k=5)
k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(vectors)

# 5. 結果をDataFrameに格納
df = pd.DataFrame({'country': valid_countries, 'cluster': clusters})

# 6. 各クラスタの国を表示
for i in range(k):
    print(f"\nクラスタ {i + 1}:")
    print(df[df['cluster'] == i]['country'].values)

モデルに存在する国数: 35

クラスタ 1:
['Canada' 'India' 'Australia' 'Nigeria' 'South_Africa' 'Pakistan'
 'Malaysia' 'Singapore']

クラスタ 2:
['Netherlands' 'Sweden' 'Norway' 'Poland' 'Denmark' 'Finland']

クラスタ 3:
['Germany' 'France' 'Italy' 'Brazil' 'Spain' 'Argentina' 'Greece'
 'Portugal']

クラスタ 4:
['Turkey' 'Egypt' 'Iran' 'Iraq' 'Israel']

クラスタ 5:
['Japan' 'China' 'Russia' 'Mexico' 'South_Korea' 'United_States' 'Vietnam'
 'Thailand']


68. Ward法によるクラスタリング<br>
国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

    memo:
    - Ward法は、階層型クラスタリング（Hierarchical Clustering）の一種で、「クラスタ内の分散を最小化しながらクラスタを統合する方法」です。

    - 階層型クラスタリングとは？
        - データポイントを階層的にクラスタリングしていく手法です。
        - 初めは各データポイントを個別のクラスタとして扱い、徐々にクラスタを統合していきます。
        - 結果として、**デンドログラム（樹形図）**が得られます。

    - Ward法の特徴
        - 統合基準：クラスタリング時に「クラスタ内誤差（分散）の増加が最も少ないクラスタを統合」します。
        - 分散最小化アプローチ：各クラスタの分散が小さいほど、クラスタの密度が高く、データのまとまりが良いと判断されます。
        - 他の手法（最短距離法、最長距離法など）よりも、均一なサイズのクラスタが得られやすいのが特徴です。

In [2]:
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 56.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=0484888376e984c633c17a9f7ad8b19b45dd4c008bc1922b89e328a7a6a97c4a
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize-matplotlib


In [3]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
import japanize_matplotlib
# Ward法による階層型クラスタリング
linkage_matrix = linkage(vectors, method='ward')

plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, labels=valid_countries, leaf_rotation=90)
plt.title('国名の階層クラスタリング（Ward法）')

NameError: name 'vectors' is not defined

69. t-SNEによる可視化<br>
ベクトル空間上の国名に関する単語ベクトルをt-SNEで可視化せよ．

In [59]:
import seaborn as sns
from sklearn.manifold import TSNE
import numpy as np

# t-SNEで2次元に次元削減
# tsne = TSNE(n_components=2, random_state=42, perplexity=int(len(valid_countries) * 0.1), n_iter=5000) # perplexity：データ数の5%〜10%が適切とされます

tsne = TSNE(n_components=2, random_state=42, perplexity=10, n_iter=5000) # perplexity：データ数の5%〜10%が適切とされます
vectors = np.array(vectors)
tsne_vectors = tsne.fit_transform(vectors)

# クラスタごとに色分けして可視化
plt.figure(figsize=(12, 8))
scatter = plt.scatter(tsne_vectors[:, 0], tsne_vectors[:, 1], c=clusters, cmap='viridis', s=250, alpha=0.5)

# 凡例（クラスタ番号）
plt.legend(*scatter.legend_elements(), title="Clusters")

# 国名をプロット
for i, country in enumerate(valid_countries):
    plt.annotate(country, (tsne_vectors[i, 0], tsne_vectors[i, 1]))

plt.title("国名のt-SNE可視化（クラスタリング付き）")

NameError: name 'vectors' is not defined